### Reducing Dimensions

We're going to look at some high dimensional data! Someone (the Broad Institute) has been kind enough to run RNA-seq on 1000 cell lines, and we are going to use PCA and UMAP to visualize expression levels of 7000 genes.

To start, we're going to import everything we need (and a few things we don't). To run the cell below, click into the cell and press `Shift+Enter`. You should see a lot of text start to appear

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
!{sys.executable} -m pip install umap-learn
import umap
from sklearn.decomposition import PCA

The next step is to read in the actual data. As is often the case with data online, they give it to you in a few pieces. Here, we are going to pull in the names of the genes and add them to the expression levels across cell lines. When you run this (click in and 'Shift+Enter'), you should see a two-row table appear below, with a lot of numbers and a lot of genes. Here every row is a cell line and every column is a gene.

In [ ]:
gene_tpm = pd.read_csv('../data/CCLE_RNAseq_rsem_genes_tpm_20180929_truncated.txt', sep='\t')
gene_names = pd.read_csv('../data/gene_mapping.csv').drop_duplicates(subset='gene_id').set_index('gene_id')
gene_data = gene_tpm.join(gene_names, on='gene_id').set_index('gene_name').iloc[:, 2:].T
gene_data['Type'] = gene_data.index.str.split('_').str[1]
gene_data.index = gene_data.index.str.split('_').str[0]
gene_data.head(2)

Next step: PCA! Principle component analysis will get you from 7000 columns to an x and a y, but it's... not exceptionally beautiful. It is very interpretable, and it will let you see some patterns, but it may not really arrange similar cell lines together. Run the two cells below to see it at work!

In [ ]:
pca = PCA()
embedding = pca.fit_transform(gene_data.iloc[:, :-1].values)
gene_data['PCA1'] = embedding[:, 0]
gene_data['PCA2'] = embedding[:, 1]


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,6), dpi=100, layout='constrained')
sns.scatterplot(data=gene_data, x='PCA1', y='PCA2', hue='Type', alpha=0.5, palette='husl')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Cancer Cell Line Type')
plt.xticks([])
plt.yticks([])
plt.show()

This is already a little interesting, we can see some cell lines are way out on the right. Let's see what we get with UMAP. UMAP and tSNE keep nearby points together, so you get a bit more of a visual representation of cluster. Notice how similar the code to make the two plots is here: we just change the transformation from `PCA` to `umap.UMAP`.

In [ ]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(gene_data.iloc[:, :-3].values)
gene_data['UMAP1'] = embedding[:, 0]
gene_data['UMAP2'] = embedding[:, 1]


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,6), dpi=100, layout='constrained')
sns.scatterplot(data=gene_data, x='UMAP1', y='UMAP2', hue='Type', alpha=0.5, palette='husl')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Cancer Cell Line Type')
plt.xticks([])
plt.yticks([])
plt.show()

That's something, at least. Now if we want to look at specific cell lines, we can overlay just the ones we like onto this same background. See below, and feel free to add another cell line or two if you like (you can use ctrl + f in the 'Cell_lines_annotations_20181227.txt' file in the data folder to check if your cell line of interest is in the dataset).

In [ ]:
# Add any cell line you like! If it's in the CCLE
cell_lines = ['HEKTE', 'HELA']
fig, ax = plt.subplots(1, 1, figsize=(8,6), dpi=100, layout='constrained')
sns.scatterplot(data=gene_data, x='UMAP1', y='UMAP2', alpha=0.5, color='grey')
sns.scatterplot(data=gene_data.loc[cell_lines, :].reset_index(), x='UMAP1', y='UMAP2', hue='index', s=100, palette='husl')
plt.legend()
plt.xticks([])
plt.yticks([])
plt.show()